In [1]:
import os
import re
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp
from collections import Counter
import seaborn as sns

In [2]:
dirs={}
dirs['project']=os.path.abspath('..')+'\\'
dirs['code']=dirs['project']+'code\\'
dirs['data']=dirs['project']+'data\\'
dirs['models']=dirs['project']+'models\\'
dirs['results']=dirs['project']+'results\\'
dirs['rawresults']=dirs['results']+'raw\\'
dirs['meltresults']=dirs['results']+'melt\\'

First, satisfy the current PowerBI design by exporting one large Destination Lookup Table

In [3]:
df_thresholds = pd.read_pickle(dirs['rawresults']+'similarity_thresholds.pkl')
df_similarity = pd.read_pickle(dirs['rawresults']+'similarity_matrix.pkl')
df_destination = pd.read_pickle(dirs['rawresults']+'destination_lookup_table.pkl')

In [4]:
df_merge = df_destination.drop(columns=['Category original']).merge(df_similarity[df_thresholds.columns], how='left', left_index=True, right_index=True, copy=True)
for t in df_thresholds.columns:
    df_merge['Yes '+t] = df_merge[t].apply(lambda s: 'Yes' if s >= df_thresholds[t].iloc[0] else 'No')
df_merge.to_excel(dirs['results']+'\\Destination lookup table.xlsx')

In [5]:
themes = df_thresholds.columns
df_thresholds = df_thresholds.transpose()
df_thresholds.index.name='Theme'
df_details = df_similarity[['Title', 'Text', 'URL']]
df_similarity = df_similarity.reset_index().melt(id_vars='Topic', var_name='Theme', value_name='Similarity', value_vars=themes).sort_values(['Topic', 'Theme']).set_index('Topic')

df_thresholds.to_csv(dirs['meltresults']+'theme_thresholds.csv')
df_similarity.to_csv(dirs['meltresults']+'topic_theme.csv')
df_destination.to_csv(dirs['meltresults']+'topic_destination.csv')
df_details.to_csv(dirs['meltresults']+'topic_details.csv')

In [6]:
df_destination

,Year,Domain,Programme,Category original,Destination / Category,Destination / Category FULL,Available Budget,Expected Grants,Joint Undertaking
Topic,,,,,,,,,
ERC-2020-PERA-2,2020,,ERC,PERA,PERA,,30000.0,,
ERC-2021-ADG,2021,,ERC,ADG,ADG,,626000000.0,,
ERC-2021-COG,2021,,ERC,COG,COG,,633000000.0,,
ERC-2021-STG,2021,,ERC,STG,STG,,619000000.0,,
ERC-2022-ADG,2022,,ERC,ADG,ADG,,471000000.0,,
...,...,...,...,...,...,...,...,...,...
DIGITAL-EUROHPC-JU-2024-NCC-02-01,2024,Digital,DIGITAL-EUROHPC-JU,NCC,SO1: HPC,High Performance Computing,5000000.0,1,EuroHPC
DIGITAL-JU-Chips-2023-SG-CPL-1,2023,Digital,DIGITAL-JU-Chips,SG-CPL,HE & DEP Other,Other actions not subject to calls for proposa...,40000000.0,3,Chips
DIGITAL-JU-Chips-2023-SG-CPL-2,2023,Digital,DIGITAL-JU-Chips,SG-CPL,HE & DEP Other,Other actions not subject to calls for proposa...,10000000.0,3,Chips
